# Team Captain - M5 Accuracy: Sales Prediction

In [0]:
import catboost

import config
import utility

In [0]:
regressor = catboost.CatBoostRegressor()
regressor.load_model(str(config.model_path / 'catboost_regressor.cbm'))

In [0]:
calendar = utility.read_calendar(config.m5_path / 'calendar.csv')

In [0]:
sales_data = utility.read_sales_data(config.m5_path / 'sales_train_validation.csv')

In [0]:
start_day = 1913  # calendar is 0-based
days = range(start_day, start_day + 28)